In [1]:
import numpy as np
from scipy import signal

In [ ]:
class ConvLayer:
    def __init__(self, n_kernels, input_shape, kernel_size):
        self.in_channels = input_shape[0]
        self.out_channels = n_kernels
        self.input_shape = input_shape
        self.kernel_size = kernel_size
        self.output_shape = (self.out_channels, input_shape[1] - kernel_size + 1, input_shape[2] - kernel_size + 1)

        if kernel_size > input_shape[1] or kernel_size > input_shape[2]:
            raise ValueError("Kernel too big for input size")
        
        self.kernels = np.random.randn(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size)
        self.biases = np.random.randn(self.out_channels)

    def forward(self, input):
        if input.shape != self.input_shape:
            raise ValueError(f"Input needs to be of shape {self.input_shape}")
        
        output = np.zeros(self.output_shape)
        
        for i in range(self.out_channels):
            for j in range(self.in_channels):
                output[i] += signal.correlate2d(input[j], self.kernels[i][j], mode='valid')
            output[i] += self.biases[i]
        
        return output
    
    def backward(self, output_grad, lr):
        kernel_grad = np.zeros(self.kernel_size, self.kernel_size)
        input_grad = np.zeros(self.input_shape)

        for i in range(self.out_channels):
            for j in range(self.in_channels):
                kernel_grad[i, j] = signal.correlate2d(output_grad[i], input[j], mode='valid')
                input_grad[j] += signal.convolve2d(output_grad[i], self.kernels[i][j], mode='full')
        
        self.kernels -= lr * kernel_grad
        self.biases -= lr * output_grad

        return input_grad

In [ ]:
class DenseLayer:
    def __init__(self, input_dim, output_dim):
        self.weights = np.random.randn(input_dim, output_dim)
        self.biases = np.zeros(output_dim)

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.biases
    
    def backward(self, output_grad, lr):
        input_grad = np.dot(output_grad, self.weights.T)

        weights_grad = np.dot(self.input.T, output_grad)


        self.weights -= lr * weights_grad
        self.biases -= lr * output_grad

        return input_grad

In [ ]:
class ReLU:
    def forward(self, input):
        self.input = input
        return np.maximum(input, 0)
    
    def backward(self, output_grad):
        return self.input * (output_grad > 0)